In [1]:
import json
import numpy
from collections import defaultdict
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import time

In [2]:
import os
import baseline
import pandas as pd
import numpy as np
#TEMPORARILY SUPPRESS WARNINGS
import warnings

warnings.filterwarnings("ignore")

# get the actual labels in the validation set

In [3]:
captions_train_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/captions_train2017.json"
captions_val_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/captions_val2017.json"
#captions_train  = json.loads(open(captions_train_path).read())

In [4]:
inst_train_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_train2017.json"
inst_val_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_val2017.json"
#inst_train  = json.loads(open(inst_train_path).read())

In [5]:
def get_electronics_info(captions_path, inst_path):
    captions  = json.loads(open(captions_path).read())
    inst  = json.loads(open(inst_path).read())
    image_df = pd.DataFrame(inst['images'])
    annotation_df = pd.DataFrame(inst['annotations'])
    #the 'id' in image_df needs to be changed to 'image_id' in order to join with annotations_df
    renamed_image_df = image_df.copy(deep=True)
    renamed_image_df.rename(columns={'id':'image_id'}, inplace=True)
    images_and_annotations_df = annotation_df.merge(renamed_image_df,on='image_id', how='left')
    electronics_only_merged_df = images_and_annotations_df.loc[(images_and_annotations_df['category_id'] >= 72) & (images_and_annotations_df['category_id'] <= 77)]
    
    #select all images that contain electronics
    #all_images_with_electronics = list(pd.Series(electronics_only_merged_df['image_id']).unique())
    
    return(electronics_only_merged_df)

In [6]:
train_electronics = get_electronics_info(captions_train_path, inst_train_path)
val_electronics = get_electronics_info(captions_val_path, inst_val_path)

In [7]:
#train_electronics.head()

In [8]:
#val_electronics.head()

In [7]:
#extract electronics IDs in each image
eIDs_in_images = val_electronics.groupby('image_id')['category_id'].apply(list).to_dict()
#select all images that contain electronics
train_images_with_electronics = list(pd.Series(train_electronics['image_id']).unique())
val_images_with_electronics = list(pd.Series(val_electronics['image_id']).unique())

In [14]:
#val_electronics

In [15]:
#val_images_with_electronics[:10]

In [16]:
len(val_images_with_electronics)

597

# get the validation images with electronics

In [8]:
val_folder_name = '/mnt/muthderd/MIDS/W210/data/val2017/'
all_val_filenames = os.listdir(val_folder_name)

In [9]:
max_file_id_length = len(all_val_filenames[0]) - 4#subtract 4 for the '.jpg' suffix

In [10]:
files_to_open = []
for image_id in val_images_with_electronics:
    file_id = str(image_id)
    zeros_to_add = max_file_id_length-len(file_id)
    filename = ('0'*zeros_to_add) + file_id + '.jpg'
    files_to_open.append(filename)

In [20]:
#files_to_open[:10]

# test models

In [11]:
%%capture

#coco_metric.reset()

#results_folder = '/mnt/muthderd/MIDS/W210/data/val_results_2/'
results_folder = '/mnt/muthderd/MIDS/W210/data/val_results/'
data_folder = '/mnt/muthderd/MIDS/W210/data/val2017/'

#test_filenames = os.listdir(data_folder)
test_filenames = files_to_open#[:25]

all_test_annotations = pd.DataFrame(columns=['image_id', 'class_id', 'scores', 1, 2, 3, 4])

#baseline_model = baseline.BaselineClassifier(model='faster_rcnn_resnet101_v1d_coco')
baseline_model = baseline.BaselineClassifier(model='yolo3_darknet53_coco')

pred_start = time.time()

for image in test_filenames:
    class_ids, scores, bounding_boxes = baseline_model.classify_objects(image, data_folder, results_folder)
    
    class_id_series = pd.Series(class_ids[0].asnumpy().reshape(len(class_ids[0]),))
    score_series = pd.Series(scores[0].asnumpy().reshape(len(scores[0]),))
    b_box_DF = pd.DataFrame(bounding_boxes[0].asnumpy().reshape(len(bounding_boxes[0]),4), columns=[1,2,3,4])
    
    class_id_series[class_id_series>-1]
    score_series[class_id_series>-1]
    b_box_DF[class_id_series>-1]
    
    test_annotations = b_box_DF[class_id_series>-1]
    test_annotations['class_id'] = class_id_series[class_id_series>-1]
    test_annotations['scores'] = score_series[class_id_series>-1]
    
    test_annotations['image_id'] = [image[:-4]]*len(test_annotations)
    test_annotations = test_annotations[['image_id', 'class_id', 'scores', 1, 2, 3, 4]]
    
    all_test_annotations = pd.concat([all_test_annotations, test_annotations], ignore_index=True)
    
    #image_id = int(image[:-4])
    #labels = val_electronics[val_electronics['image_id']==image_id]
    #gt_bboxes = np.array([[coord for coord in entry] for entry in labels['bbox']]).reshape(1,len(labels),4)
    ##gt_bboxes = np.array(labels['bbox'].apply(np.array))
    #gt_labels = np.array([entry for entry in labels['category_id']]).reshape(1,len(labels))
    ##gt_labels = np.array(labels['category_id'].apply(np.array))
    
    #map_object.update(bounding_boxes, class_ids.reshape(1,100), scores.reshape(1,100), gt_bboxes, gt_labels)
    #coco_metric.update(bounding_boxes, class_ids, scores)
    
print("Total Annotations: ", str(len(all_test_annotations)))

pred_end = time.time()
pred_duration = pred_end - pred_start

In [12]:
#map_object.get()
#coco_metric.get()

In [13]:
pred_duration

841.0118069648743

In [14]:
all_test_annotations.head()

,image_id,class_id,scores,1,2,3,4
0,000000464476,0.0,0.927936,70.859024,81.128845,162.252640,329.081268
1,000000464476,63.0,0.692063,376.342072,282.334595,660.263550,518.431458
2,000000464476,62.0,0.356848,376.498962,289.023132,657.539246,509.743530
3,000000464476,7.0,0.326432,194.305130,31.842560,372.927612,205.119370
4,000000464476,6.0,0.145672,194.305130,31.842560,372.927612,205.119370


In [15]:
#all_test_annotations.to_csv('val_result_annotations_2.csv')
all_test_annotations.to_csv('val_result_annotations.csv')

In [16]:
pred_prob_threshold = 0.5
result_annotations = all_test_annotations[all_test_annotations['scores']>pred_prob_threshold]
eIDs_in_results = result_annotations.groupby('image_id')['class_id'].apply(list).to_dict()
#eIDs_in_results

In [17]:
#quick hack to deal with the fact that the class IDs predicted by the baseline model are 10 lower than those in the annotation file
#and to throw out non electronics identified
predicted_eIDs = {}
for image_id in eIDs_in_results.keys():
    corrected_IDs = np.add(eIDs_in_results[image_id],10).astype(int)
    llimit = corrected_IDs >= 72
    ulimit = corrected_IDs <=77
    #print(llimit & ulimit)
    corrected_IDs = corrected_IDs[llimit & ulimit]
    #eIDs_in_results[image_id] = list(corrected_IDs)
    predicted_eIDs[int(image_id)] = list(corrected_IDs)
#predicted_eIDs

In [18]:
total_pred_counts = []#[0, 0, 0, 0, 0, 0]#can use a smarter structure here
total_label_counts = []#[0, 0, 0, 0, 0, 0]
#detection_accuracies = []#[0, 0, 0, 0, 0, 0]
#detection_accuracies_by_class = []
num_labels_detected = []
num_labels = []
unique_labels_by_image = []
unique_preds_by_image = []
#current counting method will not take into account false positives
for image_id in predicted_eIDs.keys():
    current_unique_preds = [0, 0, 0, 0, 0, 0]
    current_unique_labels = [0, 0, 0, 0, 0, 0]
    pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
    label_counts = [0, 0, 0, 0, 0, 0]
    
    labels = eIDs_in_images[image_id]
    for label in labels:
        label_counts[label-72] += 1
    preds = predicted_eIDs[image_id]
    for pred in preds:
        pred_counts[pred-72] += 1
    
    unique_labels = list(set(labels))
    unique_preds = list(set(preds))
    unique_label_count = len(unique_labels)
    unique_labels_detected = 0
    for label in unique_labels:
        #label_counts[label-72] += 1
        current_unique_labels[label-72] += 1
        if label in unique_preds:
            unique_labels_detected += 1
    for pred in unique_preds:
        #pred_counts[pred-72] += 1
        current_unique_preds[pred-72] += 1
    
    #new method: count instances of each id to take FP into account
    
    #new method: tally accuracy for each image separately
    
    #store counts of each label and pred for each image
    total_pred_counts.append(pred_counts)
    total_label_counts.append(label_counts)
    
    #store unique labels and preds
    unique_labels_by_image.append(current_unique_labels)
    unique_preds_by_image.append(current_unique_preds)
    
    #counts of unique labels and preds
    num_labels.append(unique_label_count)
    num_labels_detected.append(unique_labels_detected)
    
    #detection_accuracies.append(num_labels_detected/num_labels)
    #detection_accuracies_by_class.append(list(np.divide(current_pred_counts, current_label_counts)))

In [19]:
pred_counts = pd.DataFrame(total_pred_counts)
label_counts = pd.DataFrame(total_label_counts)
pred_counts[6] = pred_counts.sum(axis=1)
label_counts[6] = label_counts.sum(axis=1)

In [20]:
label_counts.head()

,0,1,2,3,4,5,6
0,2,0,0,0,0,0,2
1,0,0,0,0,0,1,1
2,0,0,0,0,0,1,1
3,1,1,2,0,1,0,5
4,0,0,0,0,1,0,1


In [21]:
errors = label_counts.subtract(pred_counts).abs()
error_rate_by_class = errors.sum(axis=0).divide(label_counts.sum(axis=0))
error_rate_by_class

0    0.431579
1    0.366071
2    0.314286
3    0.709091
4    0.543624
5    0.630522
6    0.476301
dtype: float64

In [22]:
#np.divide(num_labels_detected, num_labels)

In [23]:
#average percentage of unique electronics labels identified in a picture
np.mean(np.divide(num_labels_detected, num_labels))

0.5712962962962963

In [24]:
num_images = len(num_labels)
num_images

576

In [25]:
#which of the unique labels were predicted?
unique_labels_detected = (pd.DataFrame(unique_labels_by_image).astype(bool) & pd.DataFrame(unique_preds_by_image).astype(bool)).multiply(1).astype(int)
unique_labels_detected.sum(axis=0)

0    141
1    124
2     65
3     68
4     66
5     88
dtype: int64

In [26]:
num_images_labeled = pd.DataFrame(unique_labels_by_image).sum(axis=0)
num_images_labeled

0    204
1    176
2     87
3    138
4    103
5    205
dtype: int64

In [27]:
#overall rate of detecting that a label is present
unique_labels_detected.sum(axis=0).sum()/num_images_labeled.sum()

0.6046002190580504

In [28]:
#for each class, what fraction of images containing that class are predicted to contain that class?
unique_labels_detected.sum(axis=0).divide(num_images_labeled)

0    0.691176
1    0.704545
2    0.747126
3    0.492754
4    0.640777
5    0.429268
dtype: float64

In [ ]:
#average rate of detecting presence of a label, by class
unique_labels_detected.sum(axis=0).divide(num_images_labeled).mean()

0.6176078294692929

In [ ]:
num_images_detected = pd.DataFrame(unique_preds_by_image).sum(axis=0)
num_images_detected

0    142
1    128
2     68
3     77
4     70
5     95
dtype: int64

In [ ]:
np.divide(num_images_detected, num_images_labeled)

0    0.696078
1    0.727273
2    0.781609
3    0.557971
4    0.679612
5    0.463415
dtype: float64

In [ ]:
num_images_labeled.sum()

913

In [ ]:
%%capture

#coco_metric.reset()

results_folder = '/mnt/muthderd/MIDS/W210/data/val_results_2/'
#results_folder = '/mnt/muthderd/MIDS/W210/data/val_results/'
data_folder = '/mnt/muthderd/MIDS/W210/data/val2017/'

#test_filenames = os.listdir(data_folder)
test_filenames = files_to_open#[:25]

all_test_annotations = pd.DataFrame(columns=['image_id', 'class_id', 'scores', 1, 2, 3, 4])

baseline_model = baseline.BaselineClassifier(model='faster_rcnn_resnet101_v1d_coco')
#baseline_model = baseline.BaselineClassifier(model='yolo3_darknet53_coco')

pred_start = time.time()

for image in test_filenames:
    class_ids, scores, bounding_boxes = baseline_model.classify_objects(image, data_folder, results_folder)
    
    class_id_series = pd.Series(class_ids[0].asnumpy().reshape(len(class_ids[0]),))
    score_series = pd.Series(scores[0].asnumpy().reshape(len(scores[0]),))
    b_box_DF = pd.DataFrame(bounding_boxes[0].asnumpy().reshape(len(bounding_boxes[0]),4), columns=[1,2,3,4])
    
    class_id_series[class_id_series>-1]
    score_series[class_id_series>-1]
    b_box_DF[class_id_series>-1]
    
    test_annotations = b_box_DF[class_id_series>-1]
    test_annotations['class_id'] = class_id_series[class_id_series>-1]
    test_annotations['scores'] = score_series[class_id_series>-1]
    
    test_annotations['image_id'] = [image[:-4]]*len(test_annotations)
    test_annotations = test_annotations[['image_id', 'class_id', 'scores', 1, 2, 3, 4]]
    
    all_test_annotations = pd.concat([all_test_annotations, test_annotations], ignore_index=True)
    
    #image_id = int(image[:-4])
    #labels = val_electronics[val_electronics['image_id']==image_id]
    #gt_bboxes = np.array([[coord for coord in entry] for entry in labels['bbox']]).reshape(1,len(labels),4)
    ##gt_bboxes = np.array(labels['bbox'].apply(np.array))
    #gt_labels = np.array([entry for entry in labels['category_id']]).reshape(1,len(labels))
    ##gt_labels = np.array(labels['category_id'].apply(np.array))
    
    #map_object.update(bounding_boxes, class_ids.reshape(1,100), scores.reshape(1,100), gt_bboxes, gt_labels)
    #coco_metric.update(bounding_boxes, class_ids, scores)
    
print("Total Annotations: ", str(len(all_test_annotations)))

pred_end = time.time()
pred_duration = pred_end - pred_start

In [ ]:
pred_duration

In [ ]:
all_test_annotations.head()

In [ ]:
all_test_annotations.to_csv('val_result_annotations_2.csv')
#all_test_annotations.to_csv('val_result_annotations.csv')

In [ ]:
pred_prob_threshold = 0.5
result_annotations = all_test_annotations[all_test_annotations['scores']>pred_prob_threshold]
eIDs_in_results = result_annotations.groupby('image_id')['class_id'].apply(list).to_dict()
#eIDs_in_results

In [ ]:
#quick hack to deal with the fact that the class IDs predicted by the baseline model are 10 lower than those in the annotation file
#and to throw out non electronics identified
predicted_eIDs = {}
for image_id in eIDs_in_results.keys():
    corrected_IDs = np.add(eIDs_in_results[image_id],10).astype(int)
    llimit = corrected_IDs >= 72
    ulimit = corrected_IDs <=77
    #print(llimit & ulimit)
    corrected_IDs = corrected_IDs[llimit & ulimit]
    #eIDs_in_results[image_id] = list(corrected_IDs)
    predicted_eIDs[int(image_id)] = list(corrected_IDs)
#predicted_eIDs

In [ ]:
total_pred_counts = []#[0, 0, 0, 0, 0, 0]#can use a smarter structure here
total_label_counts = []#[0, 0, 0, 0, 0, 0]
#detection_accuracies = []#[0, 0, 0, 0, 0, 0]
#detection_accuracies_by_class = []
num_labels_detected = []
num_labels = []
unique_labels_by_image = []
unique_preds_by_image = []
#current counting method will not take into account false positives
for image_id in predicted_eIDs.keys():
    current_unique_preds = [0, 0, 0, 0, 0, 0]
    current_unique_labels = [0, 0, 0, 0, 0, 0]
    pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
    label_counts = [0, 0, 0, 0, 0, 0]
    
    labels = eIDs_in_images[image_id]
    for label in labels:
        label_counts[label-72] += 1
    preds = predicted_eIDs[image_id]
    for pred in preds:
        pred_counts[pred-72] += 1
    
    unique_labels = list(set(labels))
    unique_preds = list(set(preds))
    unique_label_count = len(unique_labels)
    unique_labels_detected = 0
    for label in unique_labels:
        #label_counts[label-72] += 1
        current_unique_labels[label-72] += 1
        if label in unique_preds:
            unique_labels_detected += 1
    for pred in unique_preds:
        #pred_counts[pred-72] += 1
        current_unique_preds[pred-72] += 1
    
    #new method: count instances of each id to take FP into account
    
    #new method: tally accuracy for each image separately
    
    #store counts of each label and pred for each image
    total_pred_counts.append(pred_counts)
    total_label_counts.append(label_counts)
    
    #store unique labels and preds
    unique_labels_by_image.append(current_unique_labels)
    unique_preds_by_image.append(current_unique_preds)
    
    #counts of unique labels and preds
    num_labels.append(unique_label_count)
    num_labels_detected.append(unique_labels_detected)
    
    #detection_accuracies.append(num_labels_detected/num_labels)
    #detection_accuracies_by_class.append(list(np.divide(current_pred_counts, current_label_counts)))

In [ ]:
pred_counts = pd.DataFrame(total_pred_counts)
label_counts = pd.DataFrame(total_label_counts)
pred_counts[6] = pred_counts.sum(axis=1)
label_counts[6] = label_counts.sum(axis=1)

In [ ]:
label_counts.head()

In [ ]:
errors = label_counts.subtract(pred_counts).abs()
error_rate_by_class = errors.sum(axis=0).divide(label_counts.sum(axis=0))
error_rate_by_class

In [ ]:
#np.divide(num_labels_detected, num_labels)

In [ ]:
#average percentage of unique electronics labels identified in a picture
np.mean(np.divide(num_labels_detected, num_labels))

In [ ]:
num_images = len(num_labels)
num_images

In [ ]:
#which of the unique labels were predicted?
unique_labels_detected = (pd.DataFrame(unique_labels_by_image).astype(bool) & pd.DataFrame(unique_preds_by_image).astype(bool)).multiply(1).astype(int)
unique_labels_detected.sum(axis=0)

In [ ]:
num_images_labeled = pd.DataFrame(unique_labels_by_image).sum(axis=0)
num_images_labeled

In [ ]:
#overall rate of detecting that a label is present
unique_labels_detected.sum(axis=0).sum()/num_images_labeled.sum()

In [ ]:
#for each class, what fraction of images containing that class are predicted to contain that class?
unique_labels_detected.sum(axis=0).divide(num_images_labeled)

In [ ]:
#average rate of detecting presence of a label, by class
unique_labels_detected.sum(axis=0).divide(num_images_labeled).mean()

In [ ]:
num_images_detected = pd.DataFrame(unique_preds_by_image).sum(axis=0)
num_images_detected

In [ ]:
np.divide(num_images_detected, num_images_labeled)

In [ ]:
num_images_labeled.sum()

In [ ]:
%%capture

results_folder = '/mnt/muthderd/MIDS/W210/data/val_results_3/'
data_folder = '/mnt/muthderd/MIDS/W210/data/val2017/'

#test_filenames = os.listdir(data_folder)
test_filenames = files_to_open#[:25]

all_test_annotations = pd.DataFrame(columns=['image_id', 'class_id', 'scores', 1, 2, 3, 4])

baseline_model = baseline.BaselineClassifier(model='ssd_512_resnet50_v1_coco')

pred_start = time.time()

for image in test_filenames:
    class_ids, scores, bounding_boxes = baseline_model.classify_objects(image, data_folder, results_folder)
    
    class_id_series = pd.Series(class_ids[0].asnumpy().reshape(len(class_ids[0]),))
    score_series = pd.Series(scores[0].asnumpy().reshape(len(scores[0]),))
    b_box_DF = pd.DataFrame(bounding_boxes[0].asnumpy().reshape(len(bounding_boxes[0]),4), columns=[1,2,3,4])
    
    class_id_series[class_id_series>-1]
    score_series[class_id_series>-1]
    b_box_DF[class_id_series>-1]
    
    test_annotations = b_box_DF[class_id_series>-1]
    test_annotations['class_id'] = class_id_series[class_id_series>-1]
    test_annotations['scores'] = score_series[class_id_series>-1]
    
    test_annotations['image_id'] = [image[:-4]]*len(test_annotations)
    test_annotations = test_annotations[['image_id', 'class_id', 'scores', 1, 2, 3, 4]]
    
    all_test_annotations = pd.concat([all_test_annotations, test_annotations], ignore_index=True)
    
print("Total Annotations: ", str(len(all_test_annotations)))

pred_end = time.time()
pred_duration = pred_end - pred_start

In [ ]:
pred_duration

In [ ]:
all_test_annotations[:10]

In [ ]:
all_test_annotations.to_csv('val_result_annotations_3.csv')

In [ ]:
pred_prob_threshold = 0.5
result_annotations = all_test_annotations[all_test_annotations['scores']>pred_prob_threshold]
eIDs_in_results = result_annotations.groupby('image_id')['class_id'].apply(list).to_dict()
#eIDs_in_results

In [ ]:
#quick hack to deal with the fact that the class IDs predicted by the baseline model are 10 lower than those in the annotation file
#and to throw out non electronics identified
predicted_eIDs = {}
for image_id in eIDs_in_results.keys():
    corrected_IDs = np.add(eIDs_in_results[image_id],10).astype(int)
    llimit = corrected_IDs >= 72
    ulimit = corrected_IDs <=77
    #print(llimit & ulimit)
    corrected_IDs = corrected_IDs[llimit & ulimit]
    #eIDs_in_results[image_id] = list(corrected_IDs)
    predicted_eIDs[int(image_id)] = list(corrected_IDs)
#predicted_eIDs

In [ ]:
total_pred_counts = []#[0, 0, 0, 0, 0, 0]#can use a smarter structure here
total_label_counts = []#[0, 0, 0, 0, 0, 0]
#detection_accuracies = []#[0, 0, 0, 0, 0, 0]
#detection_accuracies_by_class = []
num_labels_detected = []
num_labels = []
unique_labels_by_image = []
unique_preds_by_image = []
#current counting method will not take into account false positives
for image_id in predicted_eIDs.keys():
    current_unique_preds = [0, 0, 0, 0, 0, 0]
    current_unique_labels = [0, 0, 0, 0, 0, 0]
    pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
    label_counts = [0, 0, 0, 0, 0, 0]
    
    labels = eIDs_in_images[image_id]
    for label in labels:
        label_counts[label-72] += 1
    preds = predicted_eIDs[image_id]
    for pred in preds:
        pred_counts[pred-72] += 1
    
    unique_labels = list(set(labels))
    unique_preds = list(set(preds))
    unique_label_count = len(unique_labels)
    unique_labels_detected = 0
    for label in unique_labels:
        #label_counts[label-72] += 1
        current_unique_labels[label-72] += 1
        if label in unique_preds:
            unique_labels_detected += 1
    for pred in unique_preds:
        #pred_counts[pred-72] += 1
        current_unique_preds[pred-72] += 1
    
    #new method: count instances of each id to take FP into account
    
    #new method: tally accuracy for each image separately
    
    #store counts of each label and pred for each image
    total_pred_counts.append(pred_counts)
    total_label_counts.append(label_counts)
    
    #store unique labels and preds
    unique_labels_by_image.append(current_unique_labels)
    unique_preds_by_image.append(current_unique_preds)
    
    #counts of unique labels and preds
    num_labels.append(unique_label_count)
    num_labels_detected.append(unique_labels_detected)
    
    #detection_accuracies.append(num_labels_detected/num_labels)
    #detection_accuracies_by_class.append(list(np.divide(current_pred_counts, current_label_counts)))

In [ ]:
pred_counts = pd.DataFrame(total_pred_counts)
label_counts = pd.DataFrame(total_label_counts)
pred_counts[6] = pred_counts.sum(axis=1)
label_counts[6] = label_counts.sum(axis=1)

In [ ]:
label_counts.head()

In [ ]:
errors = label_counts.subtract(pred_counts).abs()
error_rate_by_class = errors.sum(axis=0).divide(label_counts.sum(axis=0))
error_rate_by_class

In [ ]:
#np.divide(num_labels_detected, num_labels)

In [ ]:
#average percentage of unique electronics labels identified in a picture
np.mean(np.divide(num_labels_detected, num_labels))

In [ ]:
num_images = len(num_labels)
num_images

In [ ]:
#which of the unique labels were predicted?
unique_labels_detected = (pd.DataFrame(unique_labels_by_image).astype(bool) & pd.DataFrame(unique_preds_by_image).astype(bool)).multiply(1).astype(int)
unique_labels_detected.sum(axis=0)

In [ ]:
num_images_labeled = pd.DataFrame(unique_labels_by_image).sum(axis=0)
num_images_labeled

In [ ]:
#overall rate of detecting that a label is present
unique_labels_detected.sum(axis=0).sum()/num_images_labeled.sum()

In [ ]:
#for each class, what fraction of images containing that class are predicted to contain that class?
unique_labels_detected.sum(axis=0).divide(num_images_labeled)

In [ ]:
#average rate of detecting presence of a label, by class
unique_labels_detected.sum(axis=0).divide(num_images_labeled).mean()

In [ ]:
num_images_detected = pd.DataFrame(unique_preds_by_image).sum(axis=0)
num_images_detected

In [ ]:
np.divide(num_images_detected, num_images_labeled)

In [ ]:
num_images_labeled.sum()